# Función de _ranking_ 1: Similaridad del coseno

Mas allá del _scoring_ _baseline_ (un tanto _naive_) anterior, la primera función de _ranking_ más elaborada que aplicaremos consistirá en una variante clásica de la similitud coseno (con la norma L1). Se trata esencialmente de construir el vector del documento y el vector de la consulta para luego tomar el producto escalar como resultado.
En realidad, es exactamente lo mismo que hemos hecho ya para el simple conteo de términos anterior (suponiendo que el vector consulta era siempre un vector binario con 1 en los términos contenidos en la consulta, y 0 en el resto). Ahora, sin embargo, en lugar de tomar simplemente los conteos de términos (tanto en la consulta como en el documento), podrían considerarse varias alternativas para calcular el peso (=componente) de cada término, decidiendo:

1. Cómo se calcula exactamente la frecuencia de cada término.
2. Cómo se realiza la ponderación por frecuencia de documento de cada término.
3. La estrategia de normalización seguida.

De nuevo, la figura 6.15 de la página 128 del libro de Manning ([enlace](http://nlp.stanford.edu/IR-book/pdf/06vect.pdf)) nos recuerda todas las posibles alternativas para ello, según la notación SMART.

En lo que sigue discutiremos las opciones para ambos vectores por separado.

## Vectores de consulta

* **Frecuencia del término** (_tf_):
Las frecuencias crudas pueden computarse de forma sencilla a partir de los términos de la consulta. Deberían corresponderse, en la mayoría de los casos, con un simple 1 para cada término que apareciese en la consulta, equivalente a la opción _"boolean"_ (pero no necesariamente, ya que algún término podría aparecer repetido). Dicha frecuencia cruda podría, si se desease, ser escalada sublinealmente (usando el logaritmo). En todo caso, en este notebook mantendremos el enfoque simple del conteo natural de términos, muy similar al vector booleano, dado que la inmensa mayoría de las consultas del _dataset_ no contienen términos repetidos.

* **Frecuencia del documento** (_df_):
Cada uno de los términos en el vector de la consulta deberá entonces ser pesado (=multiplicado) usando el valor de IDF correspondiente para cada término. Usaremos, como ya se comentó antes en este mismo notebook, el IDF computado a partir del corpus de la práctica 1. Recuérdese que, para el caso de palabras que no apareciesen en dicho corpus, se usará la técnica del suavizado Laplaciano (es decir, simplemente sumar 1 en el numerador y en el denominador; esto equivale esencialmente a asumir la existencia de un hipotético documento _"dummy"_ que contiene todos los posibles términos)

* **Normalización** (_norm_):
En ningún caso será necesario normalizar el vector de consulta, ya que cualquier posible normalización se aplicaría por igual al cruzarlo con todos los correspondientes documentos resultado, obteniéndose un simple escalado uniforme de los valores de _scoring_, lo que obviamente no influiría en absoluto en el correspondiente _ranking_ de resultados.

## Vectores de documento

* **Frecuencia del término** (_tf_):
Al igual que con el vector de consulta, podremos utilizar directamente las frecuencias crudas, o, alternativamente, aplicar algún tipo de escalado sublineal. En particular, el escalado sublineal típico es $tf_i = 1 + log(rs_i)$ si $rs_i > 0$, o simplemente $0$ en caso contrario. Así, por ejemplo, para el anterior vector _tf_ del campo **body** del documento _d_, el vector resultante sería $[\text{1+log(10)  1+log(7)  1+log(1)  0}]^T$ (de nuevo, puede encontrarse más información sobre el escalado sublineal del término _tf_ en la página 126, sección 6.4.1 del [libro de Manning](http://nlp.stanford.edu/IR-book/pdf/06vect.pdf)).

* **Frecuencia del documento** (_df_):
No utilizaremos ningún tipo de frecuencia del documento en el vector de documento. En lugar de ello, se incorporará este peso únicamente en el vector de consulta, como se describía en el apartado anterior.

* **Normalización** (_norm_):
En este caso, no podemos usar la normalización del coseno, dado que nuestros archivos de entrenamiento no proporcionan acceso al contenido del documento en sí, sino solo un resumen de campos. Por lo tanto, no sabemos ni qué otros términos aparecen, ni el recuento de los mismos, en el campo **body**. En lugar de ello, por tanto, utilizaremos la normalización de longitud. Además, dado que puede haber enormes discrepancias entre las longitudes de los diferentes campos, dividimos todos los campos por el mismo factor de normalización, la propia longitud del campo **body**. Dado, además, el hecho de que algunos documentos aparecen con una longitud de 0, una buena estrategia es, de nuevo, agregar un valor (p.e. 500), a la longitud del cuerpo de cada documento. El valor concreto a utilizar, además, puede ser también utilizado para experimentar con ésta u otras estrategias de suavizado, y observar su posible influencia en los resultados de _ranking_ obtenidos.

## Pesado relativo de los diferentes campos

Dado un documento $d$ y una consulta $q$, si $qv_q$ es el vector resultante de la consulta y $tf_{d,u}$, $tf_{d,t}$, $tf_{d,b}$, $tf_{d,h}$ y $tf_{d,a}$ son los vectores resultantes para cada uno de los campos **url**, **title**, **body**, **header** and **anchor**, respectivamente, definimos el _scoring_ global neto como (nótese que el símbolo $\cdot$ se usa en la siguiente expresión tanto para el producto escalar entre vectores como para el producto de un escalar por un vector):

$$qv_q \cdot (c_u \cdot tf_{d,u} + c_t \cdot tf_{d,t} + c_b \cdot tf_{d,b} + c_h \cdot tf_{d,h} + c_a \cdot tf_{d,a})$$

Donde $c_u$, $c_t$, $c_b$, $c_h$ y $c_a$ son los pesos dados a los campos **url**, **title**, **body**, **header** y **anchor**, respectivamente.

Por supuesto, para usar la expresión anterior necesitamos determinar de una forma sensata los pesos para cada uno de estos cinco campos. En este sentido, trataremos de escogerlos de forma que la función NDCG de evaluación (que describiremos más adelante) obtenga un valor lo más optimizado posible cuando sea aplicada al conjunto de test completo. Usaremos el conjunto de _training_ para intentar derivar dicho valor óptimo de los cinco parámetros mencionados, para luego evaluar su rendimiento en el conjunto de _test_. En una primera instancia, lo intentaremos hacer manualmente, intentando razonar simplemente sobre la importancia relativa de los diferentes pesos. Al final del notebook sustituiremos esta suerte de "razonamiento manual" por una śencilla técnica de _machine learning_.

Nótese que el valor absoluto de dichos pesos no importará, sólo la relación (ratio) entre ellos (valores relativos), dado que si multiplicamos todos los pesos por la misma constante, el _scoring_ final quedará simplemente multiplicado por dicha constante para todos los documentos por igual, lo que obviamente no afectará en absoluto a la ordenación.

### Esquema de _weighting_ inicial

Proporcionamos aquí un esquema de pesado por defecto de partida, que puede después variarse para intentar mejorar el rendimiento (medido con NDGC). Nótese que:
* En estos primeros pesos por defecto se asigna una importancia del peso de la URL 100 veces superior a la del resto de pesos, a los que, por otro lado, se da un peso equivalente.
* Se añade al conjunto de parámetros ajustables un parámetro de suavizado de la longitud del documento (`smoothing_body_length`), término que podrá ser modificado para influir en la función de _scoring_ final. Dicho término será simplemente sumado a la longitud original de cada documento (con lo que, en todo caso, y como se comentó anteriormente, se evitará siempre la posible división por cero para documentos en los cuales la longitud indicada en el documento del dataset de entrada sea cero). Se deduce, pues, que dar un valor cada vez mayor para este parámetro implicará una influencia progresivamente menor del campo `body_length` original de cada documento particular, ya que el correspondiente factor de influencia de la longitud tenderá con ello a homogeneizarse más para todos los documentos, al disminuir progresivamente el peso relativo del valor inicial de `body_length` en la suma total del denominador.

In [20]:
params_cosine = {
    "url_weight" : 10,
    "title_weight": 0.1,
    "body_hits_weight" : 0.1,
    "header_weight" : 0.1,
    "anchor_weight" : 0.1,
    "smoothing_body_length" : 500,
}

## Clase _CosineSimilarityScorer_

He aquí la definición de una clase para realizar un _scoring_ basado en la similaridad del coseno (basada en la clase `AbstractScorer` definida anteriormente, y reimplementando los métodos adecuados):

In [ ]:
class CosineSimilarityScorer(AbstractScorer):

    def __init__(self, idf, query_dict, params, query_weight_scheme=None, doc_weight_scheme=None):
        # Inicializamos clase base "AbstractScorer", ...
        super().__init__(idf, query_weight_scheme=query_weight_scheme, doc_weight_scheme=doc_weight_scheme)
        self.query_dict = query_dict
        # ... y añadimos los parámetros necesarios (5 pesos de los 5 campos + factor suavizado longitud):
        try:
            self.smoothing_body_length = params["smoothing_body_length"]
        except:
            self.smoothing_body_length = 0
        self.weights = {"url": params["url_weight"], "title": params["title_weight"],
                        "headers": params["header_weight"], "anchors": params["anchor_weight"],
                        "body_hits": params["body_hits_weight"]}

    def get_query_vector(self, q):
        """ Usando los vectores de conteo crudos de la clase base, aplica diferentes variantes
            SMART para obtener el correspondiente vector numéricos de consulta modificado.
        Args:
            q (Query): Query("Una consulta determinada")
        Returns:
            query_vec (dict): El vector resultado.
        """
        # Método de conteo de la clase base:
        query_vec = super().get_query_vector(q)

        # Frecuencia de documento (implementadas las variantes n, b, y t SMART):
        if self.query_weight_scheme["tf"] == "b":   # Vector query_vec booleano:
            ### BEGIN YOUR CODE (FIXME)
           
            ### END YOUR CODE (FIXME)
        if self.query_weight_scheme["df"] == "n":     # No se modifica query_vec:
            pass
        elif self.query_weight_scheme["df"] == "t":   # Modificación IDF de query_vec:
            ### BEGIN YOUR CODE (FIXME)
            

            ### END YOUR CODE (FIXME)
        return query_vec

    def get_doc_vector(self, q, d):
        """ Usando los vectores de conteo crudos de la clase base, aplica diferentes variantes
            SMART para obtener los correspondientes vectores numéricos de documento modificados.
        Args:
        q (Query) : Query("Una consulta")
        d (Document) : Query("Una consulta")["Un URL"]
        Returns:
        doc_vec (dict) : Vectores numéricos modificados, de nuevo con esquema (tipo_de_campo -> (término -> conteo))
                    Ejemplo: "{'url':   {'stanford': 0.13, 'aoerc': 0, 'pool': 0, 'hours': 0},
                               'title': {'stanford': 0.11, 'aoerc': 0, 'pool': 0, 'hours': 0},
                               ...
                               }"
        """
        # Método de conteo de la clase base:
        doc_vec = super().get_doc_vector(q, d)

        # Frecuencia de término (implementadas las variantes n y l SMART):
        if self.doc_weight_scheme["tf"] == "n":   # No se modifica doc_vec:
            pass
        elif self.doc_weight_scheme["tf"] == "l": # Modificación logarítmica (sublineal) de doc_vec
            ### BEGIN YOUR CODE (FIXME)
            
            ### END YOUR CODE (FIXME)
        # Normalización:
        if self.doc_weight_scheme['norm'] == "default":
            doc_vec = self.normalize_doc_vec(q, d, doc_vec)

        return doc_vec

    def get_sim_score(self, q, d, field_type):
        """ Cálculo del scoring para un campo individual:
        Args:
            q (Query) : La consulta.
            d (Document) : El documento.
            field_type (str) : El campo del que se usará el vector de documento.
        Return:
            score (float) : El scoring individual (para el campo field_type) del par (q,d):
        """
        ### BEGIN YOUR CODE (FIXME)

        ### END YOUR CODE (FIXME)
        return score

    def get_net_score(self, q, d):
        """ Cálculo del scoring global (neto), usando los cinco pesos:
        Args:
            q (Query) : La consulta.
            d (Document) : El documento.
        Return:
            score (float) : El scoring global (neto, sumando todos los campos) del par (q,d):
        """
        ### BEGIN YOUR CODE (FIXME)

        ### END YOUR CODE (FIXME)
        return score

    ## Normalización
    def normalize_doc_vec(self, q, d, doc_vec):
        """ Normalización del vector de documento:
        Damos una normalización uniforme basada en la longitud del documento, tal
        y como se discutió en el item "Normalización" del anterior apartado.
        Es decir, dividimos cada componente del vector de documento por
        (longitud_del_cuerpo_del_documento + factor_de_suavizado).
        
        Args:
            q (Query) : La consulta.
            d (Document) : El documento.
            doc_vec (dict) : El vector de documento.
        Return:
            doc_vec (dict) : El vector de documento tras la normalización.
        """
        ### BEGIN YOUR CODE (FIXME)

        ### END YOUR CODE (FIXME)
        # print(d.body_length, self.smoothing_body_length)

        return doc_vec

He aquí una primera prueba sencilla de _scoring_ de un par ($q$,$d$) usando esta similaridad del coseno, en particular usando el esquema SMART _ddd.qqq_ = _nnn.bnn_:

In [ ]:
q = Query("stanford aoerc pool hours")
d = query_dict[q]['http://events.stanford.edu/2014/February/18/']
doc_weight_scheme = {"tf": 'n', "df": 'n', "norm": None}
query_weight_scheme = {"tf": 'b', "df": 'n', "norm": None}
cs = CosineSimilarityScorer(theIDF, query_dict, params_cosine, query_weight_scheme, doc_weight_scheme)
print('Vector consulta: ', cs.get_query_vector(q), '\n')
print('Vector de documento original:\n', cs.get_doc_vector(q, d), '\n')
print("---")
print("Scoring campo url:", cs.get_sim_score(q,d,"url"))
print("Scoring campo title:", cs.get_sim_score(q,d,"title"))
print("Scoring campo headers:", cs.get_sim_score(q,d,"headers"))
print("Scoring campo anchors:", cs.get_sim_score(q,d,"anchors"))
print("Scoring campo body_hits:", cs.get_sim_score(q,d,"body_hits"))
print("\nScoring neto:", cs.get_net_score(q,d))

La salida tendría que ser la siguiente:

Y he aquí, para el mismo par ($q$,$d$), algunos posibles vectores alternativos, obtenidos usando diferentes variantes SMART, tanto para la consulta $q$ como para el documento $d$:

In [ ]:
q = Query("stanford aoerc pool hours")
d = query_dict[q]['http://events.stanford.edu/2014/February/18/']

query_weight_scheme, doc_weight_scheme = None, None

query_weight_scheme = {"tf": 'b', "df": 'n', "norm": None}
cs = CosineSimilarityScorer(theIDF, query_dict, params_cosine, query_weight_scheme, doc_weight_scheme)
print('Vector consulta original: ', cs.get_query_vector(q), '\n')

query_weight_scheme = {"tf": 'b', "df": 't', "norm": None}
cs = CosineSimilarityScorer(theIDF, query_dict, params_cosine, query_weight_scheme, doc_weight_scheme)
print('Vector consulta IDF: ', cs.get_query_vector(q), '\n')

doc_weight_scheme = {"tf": 'n', "df": 'n', "norm": None}
cs = CosineSimilarityScorer(theIDF, query_dict, params_cosine, query_weight_scheme, doc_weight_scheme)
print('Vector de documento original:\n', cs.get_doc_vector(q, d), '\n')
print("-----")

doc_weight_scheme = {"tf": 'n', "df": 'n', "norm": "default"}
cs = CosineSimilarityScorer(theIDF, query_dict, params_cosine, query_weight_scheme, doc_weight_scheme)
print('Vector de documento normalizado:\n', cs.get_doc_vector(q, d), '\n')
print("-----")

doc_weight_scheme = {"tf": 'l', "df": 'n', "norm": None}
cs = CosineSimilarityScorer(theIDF, query_dict, params_cosine, query_weight_scheme, doc_weight_scheme)
print('Vector de documento con escalado logarítmico de tf:\n', cs.get_doc_vector(q, d), '\n')
print("-----")

doc_weight_scheme = {"tf": 'l', "df": 'n', "norm": "default"}
cs = CosineSimilarityScorer(theIDF, query_dict, params_cosine, query_weight_scheme, doc_weight_scheme)
print('Vector de documento con escalado logarítmico y normalizado:\n', cs.get_doc_vector(q, d), '\n')
print("-----")

La salida debería ser la siguiente: